This exercise will require you to pull some data from the Qunadl API. Qaundl is currently the most widely used aggregator of financial market data.

As a first step, you will need to register a free account on the http://www.quandl.com website.

After you register, you will be provided with a unique API key, that you should store:

In [1]:
# Store the API key as a string - according to PEP8, constants are always named in all upper case
API_KEY = ''

Qaundl has a large number of data sources, but, unfortunately, most of them require a Premium subscription. Still, there are also a good number of free datasets.

For this mini project, we will focus on equities data from the Frankfurt Stock Exhange (FSE), which is available for free. We'll try and analyze the stock prices of a company called Carl Zeiss Meditec, which manufactures tools for eye examinations, as well as medical lasers for laser eye surgery: https://www.zeiss.com/meditec/int/home.html. The company is listed under the stock ticker AFX_X.

You can find the detailed Quandl API instructions here: https://docs.quandl.com/docs/time-series

While there is a dedicated Python package for connecting to the Quandl API, we would prefer that you use the *requests* package, which can be easily downloaded using *pip* or *conda*. You can find the documentation for the package here: http://docs.python-requests.org/en/master/ 

Finally, apart from the *requests* package, you are encouraged to not use any third party Python packages, such as *pandas*, and instead focus on what's available in the Python Standard Library (the *collections* module might come in handy: https://pymotw.com/3/collections/ ).
Also, since you won't have access to DataFrames, you are encouraged to us Python's native data structures - preferably dictionaries, though some questions can also be answered using lists.
You can read more on these data structures here: https://docs.python.org/3/tutorial/datastructures.html

Keep in mind that the JSON responses you will be getting from the API map almost one-to-one to Python's dictionaries. Unfortunately, they can be very nested, so make sure you read up on indexing dictionaries in the documentation provided above.

In [2]:
# First, import the relevant modules

import requests
import json
import operator

In [3]:
# Now, call the Quandl API and pull out a small sample of the data (only one day) to get a glimpse
# into the JSON structure that will be returned

url = 'https://www.quandl.com/api/v3/datasets/FSE/AFX_X.json?&start_date=2014-01-01&end_date=2014-01-02&api_key' + API_KEY
r = requests.get(url)
json_data = r.json()

In [4]:
# Inspect the JSON structure of the object you created, and take note of how nested it is,
# as well as the overall structure

json_data

{'dataset': {'id': 10095370,
  'dataset_code': 'AFX_X',
  'database_code': 'FSE',
  'name': 'Carl Zeiss Meditec (AFX_X)',
  'description': 'Stock Prices for Carl Zeiss Meditec (2020-11-02) from the Frankfurt Stock Exchange.<br><br>Trading System: Xetra<br><br>ISIN: DE0005313704',
  'refreshed_at': '2020-12-01T14:48:09.907Z',
  'newest_available_date': '2020-12-01',
  'oldest_available_date': '2000-06-07',
  'column_names': ['Date',
   'Open',
   'High',
   'Low',
   'Close',
   'Change',
   'Traded Volume',
   'Turnover',
   'Last Price of the Day',
   'Daily Traded Units',
   'Daily Turnover'],
  'frequency': 'daily',
  'type': 'Time Series',
  'premium': False,
  'limit': None,
  'transform': None,
  'column_index': None,
  'start_date': '2014-01-01',
  'end_date': '2014-01-02',
  'data': [['2014-01-02',
    24.24,
    24.76,
    23.77,
    24.18,
    None,
    55735.0,
    1355658.0,
    None,
    None,
    None]],
  'collapse': None,
  'order': None,
  'database_id': 6129}}

These are your tasks for this mini project:

1. Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017 (keep in mind that the date format is YYYY-MM-DD).
2. Convert the returned JSON object into a Python dictionary.
3. Calculate what the highest and lowest opening prices were for the stock in this period.
4. What was the largest change in any one day (based on High and Low price)?
5. What was the largest change between any two days (based on Closing Price)?
6. What was the average daily trading volume during this year?
7. (Optional) What was the median trading volume during this year. (Note: you may need to implement your own function for calculating the median.)

In [12]:
#Collecting data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017.

url = 'https://www.quandl.com/api/v3/datasets/FSE/AFX_X.json?&start_date=2017-01-01&end_date=2017-12-31&api_key' + API_KEY
r = requests.get(url)
json_data = r.json()

In [6]:
#Converting the returned JSON object into a Python dictionary

json_dict = dict(json_data)

In [7]:
#Calculating what the highest and lowest opening prices were for the stock in this period

column_names = json_dict['dataset']['column_names']
data_dict = json_dict['dataset']['data']

print(column_names)

['Date', 'Open', 'High', 'Low', 'Close', 'Change', 'Traded Volume', 'Turnover', 'Last Price of the Day', 'Daily Traded Units', 'Daily Turnover']


In [8]:
opening_prices = {}

date_index = column_names.index('Date')
open_index = column_names.index('Open')

for date in data_dict:
    if date[open_index] != None:
        opening_prices[date[date_index]] = date[open_index]

max_opening_price = max(opening_prices.values())
min_opening_price = min(opening_prices.values())

print('The highest opening price is', max_opening_price)
print('The lowest opening price is', min_opening_price)

The highest opening price is 53.11
The lowest opening price is 34.0


In [9]:
#Calculating the largest change in any one day (based on High and Low price)

intraday_change = {}

high_index = column_names.index('High')
low_index = column_names.index('Low')

for date in data_dict:
    intraday_change[date[date_index]] = date[high_index] - date[low_index]

max_intraday_change = round(max(intraday_change.values()),2)

print('The largest change in any one day (based on High and Low price and rounded to the nearest hundredth) is'
      , max_intraday_change)

The largest change in any one day (based on High and Low price and rounded to the nearest hundredth) is 2.81


In [10]:
#Calculating the largest change between any two days (based on Closing Price)

closing_prices = {}
interday_prices = []
interday_changes = []

close_index = column_names.index('Close')

for date in data_dict:
    closing_prices[date[date_index]] = date[close_index]

for k, v in closing_prices.items():
    interday_prices.append(closing_prices[k])
    
i = 1
while i < len(interday_prices):
    interday_changes.append(interday_prices[i] - interday_prices[i-1])
    i += 1

largest_interday_change = round(max(interday_changes),2) 

print('The largest change between any two days (based on Closing Price and rounded to the nearest hundredth) is'
      , largest_interday_change)

The largest change between any two days (based on Closing Price and rounded to the nearest hundredth) is 2.56


In [11]:
#Calculating the average daily trading volume during this year

trading_volumes = []

tradVol_index = column_names.index('Traded Volume')

for date in data_dict:
    trading_volumes.append(date[tradVol_index])

average_trading_volume = (sum(trading_volumes) / len(trading_volumes))

print('The average daily trading volume (rounded to the nearest hundredth) is'
      , round(average_trading_volume,2))

The average daily trading volume (rounded to the nearest hundredth) is 89124.34
